In [ ]:
!pip install imagehash

In [ ]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct
from sklearn.decomposition import PCA
import imagehash
import pandas as pd

# Load the image
image_path = '/content/WhatsApp Image 2024-06-08 at 15.18.54.jpeg'
image = Image.open(image_path).convert('L')  # Convert to grayscale

# Resize image for hashing methods
resize_dim = (64, 64)  # Resize to a larger dimension to have enough features for PCA
image_resized = image.resize(resize_dim, Image.Resampling.LANCZOS)

# Convert image to numpy array
pixels = np.array(image_resized).astype(float)

# Function to calculate PCA-based hash
def pca_hash(image_array, hash_size=8):
    # Apply PCA on the 2D image array directly
    pca = PCA(n_components=hash_size)
    pca.fit(image_array)
    pca_components = pca.transform(image_array)

    # Select the first few principal components to form the hash
    pca_selected = pca_components[:hash_size, :hash_size]

    # Calculate the mean of the selected components
    pca_mean = np.mean(pca_selected)

    # Generate the hash based on whether each component is above or below the mean
    hash_array = (pca_selected > pca_mean).astype(int)

    # Convert the hash array to a binary string
    hash_string = ''.join(str(x) for x in hash_array.flatten())

    # Convert binary string to hexadecimal
    hash_hex = hex(int(hash_string, 2))[2:].zfill(hash_size * hash_size // 4)

    return hash_hex

# Function to calculate DCT-based hash
def dct_hash(image_array, hash_size=8):
    dct_array = dct(dct(image_array, axis=0), axis=1)
    dct_low_freq = dct_array[:hash_size, :hash_size]
    median_val = np.median(dct_low_freq)
    hash_array = (dct_low_freq > median_val).astype(int)
    hash_string = ''.join(str(x) for x in hash_array.flatten())
    hash_hex = hex(int(hash_string, 2))[2:].zfill(hash_size * hash_size // 4)
    return hash_hex

# Calculate hashes using other perceptual hashing methods
a_hash = imagehash.average_hash(image_resized)
d_hash = imagehash.dhash(image_resized)

# Compute the PCA-based hash
pca_hash_value = pca_hash(pixels, hash_size=8)  # Using hash_size=8 for the number of components
# Compute the DCT-based hash
dct_hash_value = dct_hash(pixels, hash_size=8)

# Display the hashes
hashes = {
    "PCA-based Hash (KLT)": pca_hash_value,
    "DCT-based Hash": dct_hash_value,
    "Average Hash (aHash)": str(a_hash),
    "Difference Hash (dHash)": str(d_hash)
}

# Convert the hashes to a DataFrame for better visualization
hashes_df = pd.DataFrame.from_dict(hashes, orient='index', columns=['Hash Value'])

In [ ]:
hashes_df

,Hash Value
PCA-based Hash (KLT),a8a8a8a8a8a8aaa2
DCT-based Hash,ccb4a7f2188b334e
Average Hash (aHash),fff753db98003000
Difference Hash (dHash),0da6b6b33107e141


In [ ]:
# Install youtube-dl
!pip install youtube-dl

# # Download a video from YouTube
# !youtube-dl -f best -o sample_video.mp4 https://www.youtube.com/watch?v=OPf0YbXqDm0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.4 MB/s eta 0:00:00
[youtube] OPf0YbXqDm0: Downloading webpage
[youtube] OPf0YbXqDm0: Downloading player 4fc7f9fa
ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


In [ ]:
import numpy as np
import cv2
from PIL import Image
from sklearn.decomposition import PCA
from scipy.fftpack import dct
import imagehash
import warnings

def thph_hash(video_path, hash_size=8):
    cap = cv2.VideoCapture(video_path)
    previous_frame_hash = None
    video_hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Step 1: Convert to grayscale and resize
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (64, 64), interpolation=cv2.INTER_LANCZOS4)
        pixels = np.array(resized_frame).astype(float)

        # Step 2: Apply PCA to capture main features
        pca = PCA(n_components=hash_size)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            pca_transformed = pca.fit_transform(pixels)

        # Ensure we have valid PCA data
        if np.isnan(pca_transformed).any():
            pca_transformed = np.nan_to_num(pca_transformed)

        # Step 3: Apply DCT to PCA-transformed data
        dct_transformed = dct(dct(pca_transformed, axis=0), axis=1)
        dct_low_freq = dct_transformed[:hash_size, :hash_size]
        dct_median_val = np.median(dct_low_freq)
        dct_hash_array = (dct_low_freq > dct_median_val).astype(int)
        dct_hash_string = ''.join(str(x) for x in dct_hash_array.flatten())
        dct_hash_hex = hex(int(dct_hash_string, 2))[2:].zfill(hash_size * hash_size // 4)

        # Step 4: Apply difference hashing for temporal changes
        pil_image = Image.fromarray(resized_frame)
        current_frame_hash = imagehash.dhash(pil_image)

        if previous_frame_hash is not None:
            # Compute the difference between hashes
            diff_hash_array = np.array(current_frame_hash.hash) ^ np.array(previous_frame_hash.hash)
            temporal_diff_hash = diff_hash_array.flatten().astype(int)
        else:
            temporal_diff_hash = np.zeros((hash_size * hash_size, ), dtype=int)

        previous_frame_hash = current_frame_hash

        # Step 5: Combine PCA-DCT and Temporal dHash results
        combined_hash_array = np.concatenate([dct_hash_array.flatten(), temporal_diff_hash])
        combined_hash_string = ''.join(str(x) for x in combined_hash_array)
        combined_hash_hex = hex(int(combined_hash_string, 2))[2:].zfill(len(combined_hash_array) // 4)

        video_hashes.append(combined_hash_hex)

    cap.release()
    return video_hashes

# Example usage for a video file
video_hashes = thph_hash('/content/Mark Ronson - Uptown Funk (Official Video) ft. Bruno Mars.mp4')
print(video_hashes)


['00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', 'e8000ffce0c75f78d8dad8d828c4b0e0', '76003e8f80f8fd0f0000020262080000', 'fd00f0f38319f9430022020010000020', 'ff00f0f08c38f9c90028080802006000', 'e080f0f89cc7f3b20860000248040000', '3f003eff8c0068ef0000208404006008', 'fff80000dffe0e0620400030a4105028', 'fff00002fefc0897000000c004805000', '1f04fef0000fffd000001014a4146000', '820600f8fec7e37b000010c20a807418', '0dfcff0f11f1c00c2020220832000010', 'e8fcef0f8cc121380000000000000200', 'e6feef0704e063110000002000444200', '0ffcff0780e0231b4060000012000010', '330200fcdfc3733e0000028800400000', '000760ffdfc06e1f400000024812000a', 'ff017cf0c0dfe1400020000020020802', 'f103fbe060fff20000202020660c0000', 'ee07efe0c0fee4002010405080400000', 'c0e0011fff070efe1800104400080014', '4000fcf0c6fffdc0585050140410300c', '0000dffcc03fffe000020202da001010', 'f9ff1f0381180dc30020282802

In [ ]:
import cv2
import numpy as np

def apply_gaussian_blur(video_path, output_path, kernel_size=(5, 5)):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blurred_frame = cv2.GaussianBlur(frame, kernel_size, 0)
        out.write(blurred_frame)

    cap.release()
    out.release()

# Apply Gaussian Blur to the video
apply_gaussian_blur('input_video.mp4', 'blurred_video.mp4')


In [ ]:
def apply_sharpening(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        sharpened_frame = cv2.filter2D(frame, -1, kernel)
        out.write(sharpened_frame)

    cap.release()
    out.release()

# Apply Sharpening to the video
apply_sharpening('input_video.mp4', 'sharpened_video.mp4')


In [ ]:
def apply_smoothing(video_path, output_path, diameter=15, sigma_color=75, sigma_space=75):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        smoothed_frame = cv2.bilateralFilter(frame, diameter, sigma_color, sigma_space)
        out.write(smoothed_frame)

    cap.release()
    out.release()

# Apply Smoothing to the video
apply_smoothing('input_video.mp4', 'smoothed_video.mp4')


In [ ]:
def apply_edge_detection(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))), False)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray_frame, 100, 200)
        out.write(edges)

    cap.release()
    out.release()

# Apply Edge Detection to the video
apply_edge_detection('input_video.mp4', 'edges_video.mp4')


In [ ]:
def apply_color_filtering(video_path, output_path, lower_bound, upper_bound):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)
        filtered_frame = cv2.bitwise_and(frame, frame, mask=mask)
        out.write(filtered_frame)

    cap.release()
    out.release()

# Apply Color Filtering to the video
apply_color_filtering('input_video.mp4', 'color_filtered_video.mp4', (35, 50, 50), (85, 255, 255))


In [ ]:
def apply_compression(video_path, output_path, compression_rate=24):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, compression_rate,
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        out.write(frame)

    cap.release()
    out.release()

# Apply Compression to the video
apply_compression('input_video.mp4', 'compressed_video.mp4', compression_rate=15)


In [ ]:
def apply_rotation(video_path, output_path, angle=90):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if angle == 90:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_180)
        elif angle == 270:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        out.write(rotated_frame)

    cap.release()
    out.release()

# Apply Rotation to the video
apply_rotation('input_video.mp4', 'rotated_video_90.mp4', angle=90)


In [ ]:
def apply_scaling(video_path, output_path, new_size):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), new_size)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, new_size)
        out.write(resized_frame)

    cap.release()
    out.release()

# Apply Scaling/Resizing to the video
apply_scaling('input_video.mp4', 'resized_video.mp4', (320, 240))


In [ ]:
def apply_cropping(video_path, output_path, crop_rect):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (crop_rect[2], crop_rect[3]))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cropped_frame = frame[crop_rect[1]:crop_rect[1] + crop_rect[3], crop_rect[0]:crop_rect[0] + crop_rect[2]]
        out.write(cropped_frame)

    cap.release()
    out.release()

# Apply Cropping to the video
apply_cropping('input_video.mp4', 'cropped_video.mp4', (50, 50, 200, 200))


In [ ]:
def apply_flipping(video_path, output_path, flip_code):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        flipped_frame = cv2.flip(frame, flip_code)
        out.write(flipped_frame)

    cap.release()
    out.release()

# Apply Flipping to the video
apply_flipping('input_video.mp4', 'flipped_video_horizontal.mp4', 1)  # Horizontal flip
apply_flipping('input_video.mp4', 'flipped_video_vertical.mp4', 0)    # Vertical flip


In [ ]:
def apply_perspective_transformation(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the transformation matrix
    pts1 = np.float32([[0, 0], [width - 1, 0], [0, height - 1], [width - 1, height - 1]])
    pts2 = np.float32([[0, 0], [width * 0.8, height * 0.2], [width * 0.2, height * 0.8], [width, height]])

    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        transformed_frame = cv2.warpPerspective(frame, matrix, (width, height))
        out.write(transformed_frame)

    cap.release()
    out.release()

# Apply Perspective Transformation to the video
apply_perspective_transformation('input_video.mp4', 'perspective_transformed_video.mp4')


In [ ]:
def apply_speed_manipulation(video_path, output_path, speed_factor=0.5):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) * speed_factor
    out = cv2.VideoWriter(output_path, fourcc, fps,
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        out.write(frame)

    cap.release()
    out.release()

# Apply Speed Manipulation to the video
apply_speed_manipulation('input_video.mp4', 'slowed_down_video.mp4', speed_factor=0.5)


In [ ]:
def apply_frame_rate_conversion(video_path, output_path, new_fps):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, new_fps,
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    frame_interval = int(cap.get(cv2.CAP_PROP_FPS) / new_fps)

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

# Apply Frame Rate Conversion to the video
apply_frame_rate_conversion('input_video.mp4', 'frame_rate_converted_video.mp4', new_fps=15)


In [ ]:
def apply_frame_dropping(video_path, output_path, drop_rate=2):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS) / drop_rate,
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % drop_rate == 0:
            out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

# Apply Frame Dropping to the video
apply_frame_dropping('input_video.mp4', 'frame_dropped_video.mp4', drop_rate=2)


In [ ]:
def apply_looping(video_path, output_path, loop_count=3):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    for _ in range(loop_count):
        for frame in frames:
            out.write(frame)

    out.release()

# Apply Looping to the video
apply_looping('input_video.mp4', 'looped_video.mp4', loop_count=3)


In [ ]:
# !pip install imagehash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import cv2
from PIL import Image
from sklearn.decomposition import PCA
from scipy.fftpack import dct
import imagehash
import warnings

def thph_hash(video_path, hash_size=8):
    cap = cv2.VideoCapture(video_path)
    previous_frame_hash = None
    video_hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Step 1: Convert to grayscale and resize
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (64, 64), interpolation=cv2.INTER_LANCZOS4)
        pixels = np.array(resized_frame).astype(float)

        # Step 2: Apply PCA to capture main features
        pca = PCA(n_components=hash_size)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            pca_transformed = pca.fit_transform(pixels)

        # Ensure we have valid PCA data
        if np.isnan(pca_transformed).any():
            pca_transformed = np.nan_to_num(pca_transformed)

        # Step 3: Apply DCT to PCA-transformed data
        dct_transformed = dct(dct(pca_transformed, axis=0), axis=1)
        dct_low_freq = dct_transformed[:hash_size, :hash_size]
        dct_median_val = np.median(dct_low_freq)
        dct_hash_array = (dct_low_freq > dct_median_val).astype(int)
        dct_hash_string = ''.join(str(x) for x in dct_hash_array.flatten())
        dct_hash_hex = hex(int(dct_hash_string, 2))[2:].zfill(hash_size * hash_size // 4)

        # Step 4: Apply difference hashing for temporal changes
        pil_image = Image.fromarray(resized_frame)
        current_frame_hash = imagehash.dhash(pil_image)

        if previous_frame_hash is not None:
            # Compute the difference between hashes
            diff_hash_array = np.array(current_frame_hash.hash) ^ np.array(previous_frame_hash.hash)
            temporal_diff_hash = diff_hash_array.flatten().astype(int)
        else:
            temporal_diff_hash = np.zeros((hash_size * hash_size, ), dtype=int)

        previous_frame_hash = current_frame_hash

        # Step 5: Combine PCA-DCT and Temporal dHash results
        combined_hash_array = np.concatenate([dct_hash_array.flatten(), temporal_diff_hash])
        combined_hash_string = ''.join(str(x) for x in combined_hash_array)
        combined_hash_hex = hex(int(combined_hash_string, 2))[2:].zfill(len(combined_hash_array) // 4)

        video_hashes.append(combined_hash_hex)

    cap.release()
    return video_hashes

# Example usage for original and modified videos
original_hashes = thph_hash('input_video.mp4')
blurred_hashes = thph_hash('blurred_video.mp4')
sharpened_hashes = thph_hash('sharpened_video.mp4')
smoothed_hashes = thph_hash('smoothed_video.mp4')
edges_hashes = thph_hash('edges_video.mp4')
color_filtered_hashes = thph_hash('color_filtered_video.mp4')
compressed_hashes = thph_hash('compressed_video.mp4')
rotated_hashes = thph_hash('rotated_video_90.mp4')
resized_hashes = thph_hash('resized_video.mp4')
cropped_hashes = thph_hash('cropped_video.mp4')
flipped_horizontal_hashes = thph_hash('flipped_video_horizontal.mp4')
flipped_vertical_hashes = thph_hash('flipped_video_vertical.mp4')
perspective_transformed_hashes = thph_hash('perspective_transformed_video.mp4')
slowed_down_hashes = thph_hash('slowed_down_video.mp4')
frame_rate_converted_hashes = thph_hash('frame_rate_converted_video.mp4')
frame_dropped_hashes = thph_hash('frame_dropped_video.mp4')
looped_hashes = thph_hash('looped_video.mp4')

# Print the first few hashes for each operation for comparison
print("Original Hashes:", original_hashes[:5])
print("Blurred Hashes:", blurred_hashes[:5])
print("Sharpened Hashes:", sharpened_hashes[:5])
print("Smoothed Hashes:", smoothed_hashes[:5])
print("Edges Hashes:", edges_hashes[:5])
print("Color Filtered Hashes:", color_filtered_hashes[:5])
print("Compressed Hashes:", compressed_hashes[:5])
print("Rotated Hashes:", rotated_hashes[:5])
print("Resized Hashes:", resized_hashes[:5])
print("Cropped Hashes:", cropped_hashes[:5])
print("Flipped Horizontal Hashes:", flipped_horizontal_hashes[:5])
print("Flipped Vertical Hashes:", flipped_vertical_hashes[:5])
print("Perspective Transformed Hashes:", perspective_transformed_hashes[:5])
print("Slowed Down Hashes:", slowed_down_hashes[:5])
print("Frame Rate Converted Hashes:", frame_rate_converted_hashes[:5])
print("Frame Dropped Hashes:", frame_dropped_hashes[:5])
print("Looped Hashes:", looped_hashes[:5])


Original Hashes: []
Blurred Hashes: []
Sharpened Hashes: []
Smoothed Hashes: []
Edges Hashes: []
Color Filtered Hashes: []
Compressed Hashes: []
Rotated Hashes: []
Resized Hashes: []
Cropped Hashes: []
Flipped Horizontal Hashes: []
Flipped Vertical Hashes: []
Perspective Transformed Hashes: []
Slowed Down Hashes: []
Frame Rate Converted Hashes: []
Frame Dropped Hashes: []
Looped Hashes: []


In [ ]:
import cv2
import numpy as np

# Path to the original video
video_path = 'CGI Animated Short Film_ _Watermelon A Cautionary Tale_ by Kefei Li & Connie Qin He _ CGMeetup.mp4'

# Define function to process and save modified videos
def process_and_save_video(input_path, output_path, process_frame_func):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        processed_frame = process_frame_func(frame)
        out.write(processed_frame)

    cap.release()
    out.release()

# Define processing functions for each operation
def apply_gaussian_blur(frame, kernel_size=(5, 5)):
    return cv2.GaussianBlur(frame, kernel_size, 0)

def apply_sharpening(frame):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    return cv2.filter2D(frame, -1, kernel)

def apply_smoothing(frame, diameter=15, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(frame, diameter, sigma_color, sigma_space)

def apply_edge_detection(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_frame, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def apply_color_filtering(frame, lower_bound=(35, 50, 50), upper_bound=(85, 255, 255)):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)
    return cv2.bitwise_and(frame, frame, mask=mask)

def apply_compression(frame, quality=30):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    result, encimg = cv2.imencode('.jpg', frame, encode_param)
    return cv2.imdecode(encimg, 1)

def apply_rotation(frame, angle=90):
    if angle == 90:
        return cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        return cv2.rotate(frame, cv2.ROTATE_180)
    elif angle == 270:
        return cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        return frame

def apply_scaling(frame, new_size=(320, 240)):
    return cv2.resize(frame, new_size)

def apply_cropping(frame, crop_rect=(50, 50, 200, 200)):
    return frame[crop_rect[1]:crop_rect[1] + crop_rect[3], crop_rect[0]:crop_rect[0] + crop_rect[2]]

def apply_flipping(frame, flip_code):
    return cv2.flip(frame, flip_code)

def apply_perspective_transformation(frame):
    height, width = frame.shape[:2]
    pts1 = np.float32([[0, 0], [width - 1, 0], [0, height - 1], [width - 1, height - 1]])
    pts2 = np.float32([[0, 0], [width * 0.8, height * 0.2], [width * 0.2, height * 0.8], [width, height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(frame, matrix, (width, height))

def apply_speed_manipulation(input_path, output_path, speed_factor):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) * speed_factor
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

def apply_frame_rate_conversion(video_path, output_path, new_fps):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, new_fps,
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    frame_interval = int(cap.get(cv2.CAP_PROP_FPS) / new_fps)

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            out.write(frame)
        frame_count += 1

    cap.release()
    out.release()


def apply_frame_dropping(video_path, output_path, drop_rate=2):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS) / drop_rate,
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % drop_rate == 0:
            out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

def apply_looping(video_path, output_path, loop_count=3):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    for _ in range(loop_count):
        for frame in frames:
            out.write(frame)

    out.release()

# Process and save each modified video
process_and_save_video(video_path, 'blurred_video.mp4', apply_gaussian_blur)
process_and_save_video(video_path, 'sharpened_video.mp4', apply_sharpening)
process_and_save_video(video_path, 'smoothed_video.mp4', apply_smoothing)
process_and_save_video(video_path, 'edges_video.mp4', apply_edge_detection)
process_and_save_video(video_path, 'color_filtered_video.mp4', apply_color_filtering)
process_and_save_video(video_path, 'compressed_video.mp4', lambda frame: apply_compression(frame, quality=30))
process_and_save_video(video_path, 'rotated_video_90.mp4', lambda frame: apply_rotation(frame, angle=90))
process_and_save_video(video_path, 'resized_video.mp4', lambda frame: apply_scaling(frame, new_size=(320, 240)))
process_and_save_video(video_path, 'cropped_video.mp4', lambda frame: apply_cropping(frame, crop_rect=(50, 50, 200, 200)))
process_and_save_video(video_path, 'flipped_video_horizontal.mp4', lambda frame: apply_flipping(frame, 1))
process_and_save_video(video_path, 'flipped_video_vertical.mp4', lambda frame: apply_flipping(frame, 0))
process_and_save_video(video_path, 'perspective_transformed_video.mp4', apply_perspective_transformation)

# Speed manipulation (slow down by 50%)
apply_speed_manipulation(video_path, 'slowed_down_video.mp4', speed_factor=0.5)

# Frame rate conversion to 15 FPS
apply_frame_rate_conversion(video_path, 'frame_rate_converted_video.mp4', new_fps=15)

# Frame dropping (drop every 2nd frame)
apply_frame_dropping(video_path, 'frame_dropped_video.mp4', drop_rate=2)

# Looping the video 3 times
def loop_video(input_path, output_path, loop_count=3):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    for _ in range(loop_count):
        for frame in frames:
            out.write(frame)

    out.release()

# Apply Looping to the video
loop_video(video_path, 'looped_video.mp4', loop_count=3)


In [ ]:
!pip install imagehash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import cv2
from PIL import Image
from sklearn.decomposition import PCA
from scipy.fftpack import dct
import imagehash
import warnings

def thph_hash(video_path, hash_size=8):
    cap = cv2.VideoCapture(video_path)
    previous_frame_hash = None
    video_hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Step 1: Convert to grayscale and resize
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (64, 64), interpolation=cv2.INTER_LANCZOS4)
        pixels = np.array(resized_frame).astype(float)

        # Step 2: Apply PCA to capture main features
        pca = PCA(n_components=hash_size)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            pca_transformed = pca.fit_transform(pixels)

        # Ensure we have valid PCA data
        if np.isnan(pca_transformed).any():
            pca_transformed = np.nan_to_num(pca_transformed)

        # Step 3: Apply DCT to PCA-transformed data
        dct_transformed = dct(dct(pca_transformed, axis=0), axis=1)
        dct_low_freq = dct_transformed[:hash_size, :hash_size]
        dct_median_val = np.median(dct_low_freq)
        dct_hash_array = (dct_low_freq > dct_median_val).astype(int)
        dct_hash_string = ''.join(str(x) for x in dct_hash_array.flatten())
        dct_hash_hex = hex(int(dct_hash_string, 2))[2:].zfill(hash_size * hash_size // 4)

        # Step 4: Apply difference hashing for temporal changes
        pil_image = Image.fromarray(resized_frame)
        current_frame_hash = imagehash.dhash(pil_image)

        if previous_frame_hash is not None:
            # Compute the difference between hashes
            diff_hash_array = np.array(current_frame_hash.hash) ^ np.array(previous_frame_hash.hash)
            temporal_diff_hash = diff_hash_array.flatten().astype(int)
        else:
            temporal_diff_hash = np.zeros((hash_size * hash_size, ), dtype=int)

        previous_frame_hash = current_frame_hash

        # Step 5: Combine PCA-DCT and Temporal dHash results
        combined_hash_array = np.concatenate([dct_hash_array.flatten(), temporal_diff_hash])
        combined_hash_string = ''.join(str(x) for x in combined_hash_array)
        combined_hash_hex = hex(int(combined_hash_string, 2))[2:].zfill(len(combined_hash_array) // 4)

        video_hashes.append(combined_hash_hex)

    cap.release()
    return video_hashes

# Compute THPH hashes for each modified video
original_hashes = thph_hash(video_path)
blurred_hashes = thph_hash('blurred_video.mp4')
sharpened_hashes = thph_hash('sharpened_video.mp4')
smoothed_hashes = thph_hash('smoothed_video.mp4')
edges_hashes = thph_hash('edges_video.mp4')
color_filtered_hashes = thph_hash('color_filtered_video.mp4')
compressed_hashes = thph_hash('compressed_video.mp4')
rotated_hashes = thph_hash('rotated_video_90.mp4')
resized_hashes = thph_hash('resized_video.mp4')
cropped_hashes = thph_hash('cropped_video.mp4')
flipped_horizontal_hashes = thph_hash('flipped_video_horizontal.mp4')
flipped_vertical_hashes = thph_hash('flipped_video_vertical.mp4')
perspective_transformed_hashes = thph_hash('perspective_transformed_video.mp4')
slowed_down_hashes = thph_hash('slowed_down_video.mp4')
frame_rate_converted_hashes = thph_hash('frame_rate_converted_video.mp4')
frame_dropped_hashes = thph_hash('frame_dropped_video.mp4')
looped_hashes = thph_hash('looped_video.mp4')

# Print the first few hashes for each operation for comparison
print("Original Hashes:", original_hashes[:5])
print("Blurred Hashes:", blurred_hashes[:5])
print("Sharpened Hashes:", sharpened_hashes[:5])
print("Smoothed Hashes:", smoothed_hashes[:5])
print("Edges Hashes:", edges_hashes[:5])
print("Color Filtered Hashes:", color_filtered_hashes[:5])
print("Compressed Hashes:", compressed_hashes[:5])
print("Rotated Hashes:", rotated_hashes[:5])
print("Resized Hashes:", resized_hashes[:5])
print("Cropped Hashes:", cropped_hashes[:5])
print("Flipped Horizontal Hashes:", flipped_horizontal_hashes[:5])
print("Flipped Vertical Hashes:", flipped_vertical_hashes[:5])
print("Perspective Transformed Hashes:", perspective_transformed_hashes[:5])
print("Slowed Down Hashes:", slowed_down_hashes[:5])
print("Frame Rate Converted Hashes:", frame_rate_converted_hashes[:5])
print("Frame Dropped Hashes:", frame_dropped_hashes[:5])
print("Looped Hashes:", looped_hashes[:5])


Original Hashes: ['00000000000000000000000000000000', 'bffe0003fffc00030000006030000000', 'fffe0001fffe00010020000000000000', 'fffe0001fffe00010000000000000000', 'fffe0001fffe00010000000000000000']
Blurred Hashes: ['00000000000000000000000000000000', 'fffe0100feff01000000006030000000', 'ffff0000ffff00000000000040000000', 'ffff0000ffff00000000000000000000', 'ffff0000ffff00000020000000000000']
Sharpened Hashes: ['00000000000000000000000000000000', 'fefe0101feff01000020006030021200', '7ffe0101fefe01010000000000020200', 'fffe0101fefe00010000000040000000', 'bffe0101fffe00010000000000000000']
Smoothed Hashes: ['00000000000000000000000000000000', 'effe0003fffc00030000006030000000', 'fffc0003fffc00030020000000000000', 'fffc0003fffc00030000000000000000', 'fffc0003fffc00030000000000000000']
Edges Hashes: ['00000000000000000000000000000000', 'ffef0400f9ff06000020007070021200', 'deff0600f9ff06000000000000020200', '5fff0600f9ff06000000000000000000', 'dbff0600f9ff06000000000000000000']
Color Filtere

In [ ]:
def apply_rotation(video_path, output_path, angle=90):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Check frame dimensions and calculate new dimensions for rotated frames
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if angle in [90, 270]:
        new_width, new_height = height, width
    else:
        new_width, new_height = width, height

    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (new_width, new_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if angle == 90:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_180)
        elif angle == 270:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            rotated_frame = frame

        out.write(rotated_frame)

    cap.release()
    out.release()

# Apply Rotation to the video (90° clockwise)
apply_rotation(video_path, 'rotated_video_90.mp4', angle=90)


In [ ]:
def apply_scaling(video_path, output_path, new_size=(320, 240)):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), new_size)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, new_size)
        out.write(resized_frame)

    cap.release()
    out.release()

# Apply Scaling/Resizing to the video
apply_scaling(video_path, 'resized_video.mp4', (320, 240))


In [ ]:
def apply_cropping(video_path, output_path, crop_rect=(50, 50, 200, 200)):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # New dimensions after cropping
    crop_width = crop_rect[2]
    crop_height = crop_rect[3]

    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (crop_width, crop_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cropped_frame = frame[crop_rect[1]:crop_rect[1] + crop_rect[3], crop_rect[0]:crop_rect[0] + crop_rect[2]]
        out.write(cropped_frame)

    cap.release()
    out.release()

# Apply Cropping to the video
apply_cropping(video_path, 'cropped_video.mp4', crop_rect=(50, 50, 200, 200))


In [ ]:
def loop_video(input_path, output_path, loop_count=3):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for _ in range(loop_count):
        for frame in frames:
            out.write(frame)

    out.release()

# Apply Looping to the video (repeat 3 times)
loop_video(video_path, 'looped_video.mp4', loop_count=3)


In [ ]:
# Compute THPH hashes for each modified video
original_hashes = thph_hash(video_path)
blurred_hashes = thph_hash('blurred_video.mp4')
sharpened_hashes = thph_hash('sharpened_video.mp4')
smoothed_hashes = thph_hash('smoothed_video.mp4')
edges_hashes = thph_hash('edges_video.mp4')
color_filtered_hashes = thph_hash('color_filtered_video.mp4')
compressed_hashes = thph_hash('compressed_video.mp4')
rotated_hashes = thph_hash('rotated_video_90.mp4')
resized_hashes = thph_hash('resized_video.mp4')
cropped_hashes = thph_hash('cropped_video.mp4')
flipped_horizontal_hashes = thph_hash('flipped_video_horizontal.mp4')
flipped_vertical_hashes = thph_hash('flipped_video_vertical.mp4')
perspective_transformed_hashes = thph_hash('perspective_transformed_video.mp4')
slowed_down_hashes = thph_hash('slowed_down_video.mp4')
frame_rate_converted_hashes = thph_hash('frame_rate_converted_video.mp4')
frame_dropped_hashes = thph_hash('frame_dropped_video.mp4')
looped_hashes = thph_hash('looped_video.mp4')

# Print the first few hashes for each operation for comparison
print("Original Hashes:", original_hashes[:5])
print("Blurred Hashes:", blurred_hashes[:5])
print("Sharpened Hashes:", sharpened_hashes[:5])
print("Smoothed Hashes:", smoothed_hashes[:5])
print("Edges Hashes:", edges_hashes[:5])
print("Color Filtered Hashes:", color_filtered_hashes[:5])
print("Compressed Hashes:", compressed_hashes[:5])
print("Rotated Hashes:", rotated_hashes[:5])
print("Resized Hashes:", resized_hashes[:5])
print("Cropped Hashes:", cropped_hashes[:5])
print("Flipped Horizontal Hashes:", flipped_horizontal_hashes[:5])
print("Flipped Vertical Hashes:", flipped_vertical_hashes[:5])
print("Perspective Transformed Hashes:", perspective_transformed_hashes[:5])
print("Slowed Down Hashes:", slowed_down_hashes[:5])
print("Frame Rate Converted Hashes:", frame_rate_converted_hashes[:5])
print("Frame Dropped Hashes:", frame_dropped_hashes[:5])
print("Looped Hashes:", looped_hashes[:5])


Original Hashes: ['00000000000000000000000000000000', 'bffe0003fffc00030000006030000000', 'fffe0001fffe00010020000000000000', 'fffe0001fffe00010000000000000000', 'fffe0001fffe00010000000000000000']
Blurred Hashes: ['00000000000000000000000000000000', 'fffe0100feff01000000006030000000', 'ffff0000ffff00000000000040000000', 'ffff0000ffff00000000000000000000', 'ffff0000ffff00000020000000000000']
Sharpened Hashes: ['00000000000000000000000000000000', 'fefe0101feff01000020006030021200', '7ffe0101fefe01010000000000020200', 'fffe0101fefe00010000000040000000', 'bffe0101fffe00010000000000000000']
Smoothed Hashes: ['00000000000000000000000000000000', 'effe0003fffc00030000006030000000', 'fffc0003fffc00030020000000000000', 'fffc0003fffc00030000000000000000', 'fffc0003fffc00030000000000000000']
Edges Hashes: ['00000000000000000000000000000000', 'ffef0400f9ff06000020007070021200', 'deff0600f9ff06000000000000020200', '5fff0600f9ff06000000000000000000', 'dbff0600f9ff06000000000000000000']
Color Filtere

In [ ]:
def hex_to_bin(hex_str):
    return bin(int(hex_str, 16))[2:].zfill(256)  # Convert hex to binary and pad to 256 bits

def hamming_distance(bin_str1, bin_str2):
    return sum(c1 != c2 for c1, c2 in zip(bin_str1, bin_str2))

def similarity_score(original_hashes, edited_hashes):
    scores = []
    for original, edited in zip(original_hashes, edited_hashes):
        bin_original = hex_to_bin(original)
        bin_edited = hex_to_bin(edited)
        distance = hamming_distance(bin_original, bin_edited)
        score = 1 - (distance / len(bin_original))
        scores.append(score)
    return scores

# Original hashes
original_hashes = ['00000000000000000000000000000000', 'bffe0003fffc00030000006030000000', 'fffe0001fffe00010020000000000000', 'fffe0001fffe00010000000000000000', 'fffe0001fffe00010000000000000000']

# Edited video hashes
blurred_hashes = ['00000000000000000000000000000000', 'fffe0100feff01000000006030000000', 'ffff0000ffff00000000000040000000', 'ffff0000ffff00000000000000000000', 'ffff0000ffff00000020000000000000']
sharpened_hashes = ['00000000000000000000000000000000', 'fefe0101feff01000020006030021200', '7ffe0101fefe01010000000000020200', 'fffe0101fefe00010000000040000000', 'bffe0101fffe00010000000000000000']
smoothed_hashes = ['00000000000000000000000000000000', 'effe0003fffc00030000006030000000', 'fffc0003fffc00030020000000000000', 'fffc0003fffc00030000000000000000', 'fffc0003fffc00030000000000000000']
edges_hashes = ['00000000000000000000000000000000', 'ffef0400f9ff06000020007070021200', 'deff0600f9ff06000000000000020200', '5fff0600f9ff06000000000000000000', 'dbff0600f9ff06000000000000000000']
color_filtered_hashes = ['00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000']
compressed_hashes = ['00000000000000000000000000000000', 'fffe0001fffc00030000006030000200', 'fffe0001fffe00010020000000000200', 'fffc0003fffc00030000000000000000', 'fffe0001fffe00010000000040000000']
rotated_hashes = ['00000000000000000000000000000000', '83fe0000ffff3e010000105030104000', '30fe0001fffe3f010000000000004000', 'c0fe0001fffe3f010000000000000000', '30ff0000ffff3e010000000000000000']
resized_hashes = ['00000000000000000000000000000000', 'fffe0101feff00000000006030000000', 'fffe0101fefe00010020000000000000', 'fefe0101fefe01010000000000000000', 'fffe0101fefe00010000000040000000']
cropped_hashes = ['00000000000000000000000000000000', '00ff0000ffffff000000060600000000', '00ff0000ffffff000000000000020000', '00ff0000ffffff000000000000000000', '80fe0000ffffff000280000000000000']
flipped_horizontal_hashes = ['00000000000000000000000000000000', 'fffe0001feff00010000007830000000', 'fffc0003fffc00030008000000000000', 'fffc0003fffc00030018000000000000', 'fffe0001fffe00010000000000000000']
flipped_vertical_hashes = ['00000000000000000000000000000000', 'ff0300fcff0300fc0002003060000000', 'ff0300fcff0300fc0002000000002000', 'ff0300fcff0300fc0000000000000000', 'ff0100feff0100fe0000000000000000']
perspective_transformed_hashes = ['00000000000000000000000000000000', 'fb6e0301fcff07000000003030000200', '3eff0700f8ff07000000004000000200', 'f3ff0600f8ff07000000000000000000', 'faff0600f8ff07000000000000040000']
slowed_down_hashes = ['00000000000000000000000000000000', 'fffe0001feff00010000006030000000', 'fffc0003fffc00030020000000000000', 'fffc0003fffc00030000000000000000', 'fffe0001fffe00010000000000000000']
frame_rate_converted_hashes = ['00000000000000000000000000000000', 'fffe0001feff00010000006030000000', 'fffc0003fffc00030020000000000000', 'fffc0003fffc00030000000000000000', 'fffe0001fffe00010000000000000000']
frame_dropped_hashes = ['00000000000000000000000000000000', 'fffe0001fffe00010020006030000000', 'fffe0001fffe00010000000000000000', 'fffe0001fffe00010000000000000000', 'fffe0001fffe00010000000000000000']
looped_hashes = ['00000000000000000000000000000000', 'fffe0001feff00010000006030000000', 'fffc0003fffc00030020000000000000', 'fffc0003fffc00030000000000000000', 'fffe0001fffe00010000000000000000']

# Calculate similarity scores
blurred_similarity = similarity_score(original_hashes, blurred_hashes)
sharpened_similarity = similarity_score(original_hashes, sharpened_hashes)
smoothed_similarity = similarity_score(original_hashes, smoothed_hashes)
edges_similarity = similarity_score(original_hashes, edges_hashes)
color_filtered_similarity = similarity_score(original_hashes, color_filtered_hashes)
compressed_similarity = similarity_score(original_hashes, compressed_hashes)
rotated_similarity = similarity_score(original_hashes, rotated_hashes)
resized_similarity = similarity_score(original_hashes, resized_hashes)
cropped_similarity = similarity_score(original_hashes, cropped_hashes)
flipped_horizontal_similarity = similarity_score(original_hashes, flipped_horizontal_hashes)
flipped_vertical_similarity = similarity_score(original_hashes, flipped_vertical_hashes)
perspective_transformed_similarity = similarity_score(original_hashes, perspective_transformed_hashes)
slowed_down_similarity = similarity_score(original_hashes, slowed_down_hashes)
frame_rate_converted_similarity = similarity_score(original_hashes, frame_rate_converted_hashes)
frame_dropped_similarity = similarity_score(original_hashes, frame_dropped_hashes)
looped_similarity = similarity_score(original_hashes, looped_hashes)

# Print similarity scores for each operation
print("Blurred Similarity Scores:", blurred_similarity)
print("Sharpened Similarity Scores:", sharpened_similarity)
print("Smoothed Similarity Scores:", smoothed_similarity)
print("Edges Similarity Scores:", edges_similarity)
print("Color Filtered Similarity Scores:", color_filtered_similarity)
print("Compressed Similarity Scores:", compressed_similarity)
print("Rotated Similarity Scores:", rotated_similarity)
print("Resized Similarity Scores:", resized_similarity)
print("Cropped Similarity Scores:", cropped_similarity)
print("Flipped Horizontal Similarity Scores:", flipped_horizontal_similarity)
print("Flipped Vertical Similarity Scores:", flipped_vertical_similarity)
print("Perspective Transformed Similarity Scores:", perspective_transformed_similarity)
print("Slowed Down Similarity Scores:", slowed_down_similarity)
print("Frame Rate Converted Similarity Scores:", frame_rate_converted_similarity)
print("Frame Dropped Similarity Scores:", frame_dropped_similarity)
print("Looped Similarity Scores:", looped_similarity)


Blurred Similarity Scores: [1.0, 0.9609375, 0.9765625, 0.984375, 0.98046875]
Sharpened Similarity Scores: [1.0, 0.9453125, 0.97265625, 0.98828125, 0.9921875]
Smoothed Similarity Scores: [1.0, 0.9921875, 0.984375, 0.984375, 0.984375]
Edges Similarity Scores: [1.0, 0.921875, 0.94140625, 0.953125, 0.953125]
Color Filtered Similarity Scores: [1.0, 0.859375, 0.87109375, 0.875, 0.875]
Compressed Similarity Scores: [1.0, 0.98828125, 0.99609375, 0.984375, 0.99609375]
Rotated Similarity Scores: [1.0, 0.92578125, 0.9453125, 0.953125, 0.9453125]
Resized Similarity Scores: [1.0, 0.96875, 0.9921875, 0.984375, 0.98828125]
Cropped Similarity Scores: [1.0, 0.8828125, 0.9140625, 0.921875, 0.921875]
Flipped Horizontal Similarity Scores: [1.0, 0.96875, 0.9765625, 0.9765625, 1.0]
Flipped Vertical Similarity Scores: [1.0, 0.85546875, 0.87890625, 0.890625, 0.875]
Perspective Transformed Similarity Scores: [1.0, 0.92578125, 0.92578125, 0.9453125, 0.94140625]
Slowed Down Similarity Scores: [1.0, 0.9765625, 0.

In [10]:
video_path = '/content/Laura vs Reavers - Fight Scene _ Logan (2017) Movie Clip HD 4K.mp4'

In [11]:
import cv2
import numpy as np

def apply_translation(video_path, output_path, tx=30, ty=30):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    # Define translation matrix
    M = np.float32([[1, 0, tx], [0, 1, ty]])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the translation
        translated_frame = cv2.warpAffine(frame, M, (width, height))
        out.write(translated_frame)

    cap.release()
    out.release()

# Apply Translation to the video (30 pixels right and down)
apply_translation(video_path, 'translated_video.mp4', tx=30, ty=30)


In [12]:
def apply_zoom(video_path, output_path, zoom_factor=1.5):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate center crop dimensions
        center_x, center_y = width // 2, height // 2
        crop_width, crop_height = int(width / zoom_factor), int(height / zoom_factor)
        crop_x1 = max(0, center_x - crop_width // 2)
        crop_x2 = min(width, center_x + crop_width // 2)
        crop_y1 = max(0, center_y - crop_height // 2)
        crop_y2 = min(height, center_y + crop_height // 2)

        # Crop and resize to original size
        cropped_frame = frame[crop_y1:crop_y2, crop_x1:crop_x2]
        zoomed_frame = cv2.resize(cropped_frame, (width, height))

        out.write(zoomed_frame)

    cap.release()
    out.release()

# Apply Zoom to the video (1.5x zoom)
apply_zoom(video_path, 'zoomed_video.mp4', zoom_factor=1.5)


In [13]:
def apply_affine_transformation(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    # Define affine transformation matrix
    pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
    pts2 = np.float32([[10, 100], [200, 50], [100, 250]])
    M = cv2.getAffineTransform(pts1, pts2)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the affine transformation
        affine_transformed_frame = cv2.warpAffine(frame, M, (width, height))
        out.write(affine_transformed_frame)

    cap.release()
    out.release()

# Apply Affine Transformation to the video
apply_affine_transformation(video_path, 'affine_transformed_video.mp4')


In [15]:
#  !pip install imagehash

In [16]:
import numpy as np
import cv2
from PIL import Image
from sklearn.decomposition import PCA
from scipy.fftpack import dct
import imagehash
import warnings

def thph_hash(video_path, hash_size=8):
    cap = cv2.VideoCapture(video_path)
    previous_frame_hash = None
    video_hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Step 1: Convert to grayscale and resize
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (64, 64), interpolation=cv2.INTER_LANCZOS4)
        pixels = np.array(resized_frame).astype(float)

        # Step 2: Apply PCA to capture main features
        pca = PCA(n_components=hash_size)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            pca_transformed = pca.fit_transform(pixels)

        # Ensure we have valid PCA data
        if np.isnan(pca_transformed).any():
            pca_transformed = np.nan_to_num(pca_transformed)

        # Step 3: Apply DCT to PCA-transformed data
        dct_transformed = dct(dct(pca_transformed, axis=0), axis=1)
        dct_low_freq = dct_transformed[:hash_size, :hash_size]
        dct_median_val = np.median(dct_low_freq)
        dct_hash_array = (dct_low_freq > dct_median_val).astype(int)
        dct_hash_string = ''.join(str(x) for x in dct_hash_array.flatten())
        dct_hash_hex = hex(int(dct_hash_string, 2))[2:].zfill(hash_size * hash_size // 4)

        # Step 4: Apply difference hashing for temporal changes
        pil_image = Image.fromarray(resized_frame)
        current_frame_hash = imagehash.dhash(pil_image)

        if previous_frame_hash is not None:
            # Compute the difference between hashes
            diff_hash_array = np.array(current_frame_hash.hash) ^ np.array(previous_frame_hash.hash)
            temporal_diff_hash = diff_hash_array.flatten().astype(int)
        else:
            temporal_diff_hash = np.zeros((hash_size * hash_size, ), dtype=int)

        previous_frame_hash = current_frame_hash

        # Step 5: Combine PCA-DCT and Temporal dHash results
        combined_hash_array = np.concatenate([dct_hash_array.flatten(), temporal_diff_hash])
        combined_hash_string = ''.join(str(x) for x in combined_hash_array)
        combined_hash_hex = hex(int(combined_hash_string, 2))[2:].zfill(len(combined_hash_array) // 4)

        video_hashes.append(combined_hash_hex)

    cap.release()
    return video_hashes


In [17]:
original_hashes = thph_hash(video_path)

def hex_to_bin(hex_str):
    return bin(int(hex_str, 16))[2:].zfill(256)  # Convert hex to binary and pad to 256 bits

def hamming_distance(bin_str1, bin_str2):
    return sum(c1 != c2 for c1, c2 in zip(bin_str1, bin_str2))

def similarity_score(original_hashes, edited_hashes):
    scores = []
    for original, edited in zip(original_hashes, edited_hashes):
        bin_original = hex_to_bin(original)
        bin_edited = hex_to_bin(edited)
        distance = hamming_distance(bin_original, bin_edited)
        score = 1 - (distance / len(bin_original))
        scores.append(score)
    return scores

In [18]:
# Function to compute similarity scores
def compute_similarity_scores(original_hashes, modified_hashes):
    scores = similarity_score(original_hashes, modified_hashes)
    average_score = sum(scores) / len(scores)
    return scores, average_score

# Compute the THPH hashes for each modified video
translated_hashes = thph_hash('translated_video.mp4')
zoomed_hashes = thph_hash('zoomed_video.mp4')
affine_transformed_hashes = thph_hash('affine_transformed_video.mp4')

# Compute and print similarity scores
translated_similarity, avg_translated_similarity = compute_similarity_scores(original_hashes, translated_hashes)
zoomed_similarity, avg_zoomed_similarity = compute_similarity_scores(original_hashes, zoomed_hashes)
affine_transformed_similarity, avg_affine_similarity = compute_similarity_scores(original_hashes, affine_transformed_hashes)

print("Translated Similarity Scores:", translated_similarity)
print("Average Translated Similarity:", avg_translated_similarity)

print("Zoomed Similarity Scores:", zoomed_similarity)
print("Average Zoomed Similarity:", avg_zoomed_similarity)

print("Affine Transformed Similarity Scores:", affine_transformed_similarity)
print("Average Affine Transformed Similarity:", avg_affine_similarity)


Translated Similarity Scores: [0.9140625, 0.93359375, 0.91015625, 0.890625, 0.90625, 0.91796875, 0.90625, 0.90234375, 0.91015625, 0.90234375, 0.88671875, 0.92578125, 0.90234375, 0.90234375, 0.89453125, 0.890625, 0.91796875, 0.91015625, 0.90625, 0.90234375, 0.8984375, 0.8828125, 0.90234375, 0.88671875, 0.87890625, 0.890625, 0.8984375, 0.89453125, 0.8828125, 0.89453125, 0.8046875, 0.88671875, 0.86328125, 0.9140625, 0.87890625, 0.8828125, 0.8828125, 0.85546875, 0.8828125, 0.87109375, 0.87109375, 0.875, 0.890625, 0.859375, 0.87109375, 0.890625, 0.87109375, 0.8828125, 0.90234375, 0.88671875, 0.875, 0.89453125, 0.875, 0.8828125, 0.90234375, 0.8984375, 0.91015625, 0.87109375, 0.83203125, 0.91796875, 0.8984375, 0.90625, 0.88671875, 0.890625, 0.90234375, 0.90625, 0.9140625, 0.796875, 0.8984375, 0.89453125, 0.8984375, 0.88671875, 0.88671875, 0.8984375, 0.8984375, 0.8984375, 0.89453125, 0.890625, 0.90625, 0.875, 0.8828125, 0.8984375, 0.89453125, 0.8984375, 0.90234375, 0.8984375, 0.89453125, 0.886

In [19]:
# prompt: make the above output into latex table ,give only average value and make table

import tabulate

# Create data for the table
data = [
    ['Translated', avg_translated_similarity],
    ['Zoomed', avg_zoomed_similarity],
    ['Affine Transformed', avg_affine_similarity],
]

# Create the table
table = tabulate.tabulate(data, headers=['Operation', 'Average Similarity'], tablefmt='latex')

# Print the table
print(table)


\begin{tabular}{lr}
\hline
 Operation          &   Average Similarity \\
\hline
 Translated         &             0.871166 \\
 Zoomed             &             0.840907 \\
 Affine Transformed &             0.848529 \\
\hline
\end{tabular}
